# NVIDIA AI Foundation Models Performance Analysis

The Nvidia **foundational Large Language Models** (LLMs) are hosted on the catalog.ngc.nvidia.com webpage.
Nvidia provides these models through either streaming or non-streaming process APIs for anyone
to interact with many models. Nvidia provides an API key to developers to interact with these 
LLMs without a cost such as the OPENAI API and the rate-limiting of OPENAI. This allows new developers to
experiment with LLMs with limited knowledge of the underlying architecture to obtain responses from 
prompts supplied.

In this notebook, we will look at 11 models that are labeled text-to-text models on their catalog. Since
all the APIs for the models are the same except the last path item which is the uuid for the model. These APIs
will be called using the Python requests module. Other request modules such as async with aiohttp, 
and httpx.

# Three main sections
1.  Run a set of prompts through a set of models and collect statistics data (non-prompt type specific).
2.  Run a set of 4 specific prompt scenarios (variation of input and output lengths) against all models and profile response behavior.
3.  Run a Gradio interface to send a query to specified models chosen in the UI. 


#### NOTE
There was experimentation with aiohttp with asyncio with the API calls. There are a few issues discovered:

    1. Some models will return a 200 status code instead of the expected 202. This required a rewrite to account
       for this problem.
       
    2. This problem is variable as it is not consistent with a particular model. If you want to async
       call the API for each model for a single prompt, one to a few of the models will return with an empty
       response. The amount of models that return an empty response will vary with each call.


In [ ]:
import requests
import os, sys
import time
import json
import yaml
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import pandas as pd 
import gradio as gr
from utils.llm_requests import ngc_request, ModelConfig, invoke_one_model, invoke_all_models
from utils.format_response import format_resp_single, format_resp_all

### Initialization 
In the cell below we will initialize our global variables that will be used by the LLM functions.

#### NOTE:
At the time of making this notebook, Mamba-chat had been released to the Nvidia catalog page on 02/12/2024.
It was not included in the set of testing due to the payload differing in structure from the other which will
require a rework to include.


In [ ]:
# NGC Model file name (base folder should contain prompt csv, llm_models_file, and ngc_api_file.
base_file_path = './data/'          # Folder that contains the below files
llm_models_file = 'llm_models.yml'  # File containing all model names
ngc_api_file = 'ngc-api.yml'        # File containing ngc url + key info
csv_prompt_file = 'questions.csv'   # All prompts for section 1 
json_prompt_file = 'kv_cache_test.json' # File that stores scenario prompts for section 2

In [ ]:
# Loading the llm_models data
with open(base_file_path + llm_models_file,'r') as f:
    model_data = yaml.load(f, Loader=yaml.SafeLoader)


In [ ]:
# Run this cell if a YAML file exists that contains your API key to NGC. 
# Read in the NGC API file. One can supplement their file in their data directory. 
with open(base_file_path + ngc_api_file, 'r') as file:
    api_var = yaml.load(file, Loader = yaml.SafeLoader)

In [ ]:
# Debugging to inspect what type the data read from the yaml files.
print(f"model_data is a {type(model_data)}, and api_var is a {type(api_var)}")

In [ ]:
# Initialize model URLs' dictionaries, model names, invoke URLs and API Key
model_urls = model_data['model_urls']
model_full_name = model_data['model_full_names'] # This variable is used for Gradio.
fetch_url_format = model_data['invoke-urls'][1]
my_api = api_var['NGCKEY']['API']

# One can also initialize their API key below by uncommenting the code below:
# os.environ['ngc_api'] = "your-key-here"
# my_api = os.environ.get['ngc_api']

In [ ]:
# debug
# print(model_urls)
# print(10*'-')
# print(fetch_url_format)
# print(my_api)

### LLM Functions
Two main LLM functions interact with the Nvidia LLMs. All the models
are interfacing with Nvidia Triton. Some of the models indicate what GPU/GPUS are
used for the interfacing while some indicate others. Comparisons may not be "apple-to-apple"
due to not all models interfacing with the same hardware.

One function works by invoking a model name and a prompt to return the model
response. This works with a helper function that formats the response to be displayed in a more readable format.
This is all done by initializing the values into a class name "ModelConfig", model name, and model dictionary.

The second function is similar to the first model saves the output in the list of dictionaries with a
similar human-readable format. 

#### Testing the LLM invocation model
In the next two cells below, one can call the function by supplying a model and prompt. The second cell
allows the user just to specify a prompt and make the call to all models in the model dictionary.

In [ ]:
# Test the invocation of the NGC API to the models
test_str = "Describe the company, Nvidia."

# Do one model call to test the connection
test_model_name = "nv-llama2-70b-rlhf"

# This class contains the prompt, API, temperature, top_p, max_tokens, seed, and stream configurations.
test_model_config = ModelConfig(test_str, my_api)

In [ ]:
# Call the model that uses a single model and prompt. Time the response.
s1_time = time.time()
llm_response = format_resp_single(test_model_name, test_model_config, model_urls)
s2_time = time.time()

print("Total time taken: {} seconds. Note actual response time from AI foundations is used in analysis \
        \n".format(round((s2_time - s1_time),3)))
print(llm_response)

#### Doing a test run of a single prompt through all models specified in YAML file


In [ ]:
# Call the function that uses a single prompt that iterates through all models. Time the response.
s3_time = time.time()
llm_all_resp = format_resp_all(test_model_config, model_urls)
s4_time = time.time()

print("Total time taken: {} minutes\n ".format(round((s4_time-s3_time) / 60,3)))
print(llm_all_resp)

# 1. Creating benchmark datasets - collect results for all models over all prompts

The section below creates the LLM results dataframes and LLM benchmark results.

The function **create_llm_dataset** generates a pandas dataframe when provided a csv or json file 
containing the input_prompt and model configurations for the eleven (11) text-to-text models.

The function **create_stats_llm** generates a pandas dataframe from the dataframe created by
create_llm_dataset that contains the statistical info of each model concerning response time
and output_tokens.

The function **df_save** saves the files created from the two functions above to a specified folder 
as json files.

In [ ]:
def create_llm_dataset(file_path:str, model_urls:dict) -> pd.DataFrame: 
    """
    Creates a data frame that is the product of running multiple
    prompts through all the models and returns the response
    message, model response time, and output tokens.

    Args:
        file_path (str): CSV | JSON file path prompts and llm model configurations.

    Outputs:
        df_llm (pd.DataFrame): Output data frame of all prompts apply.
                               to the function of calling all LLM models.

    """
    # Read in CSV prompt data and convert to list to process
    if file_path.endswith('.csv'):
        df_prompts = pd.read_csv(file_path, sep=",")
    elif file_path.endswith('.json'):
        df_prompts = pd.read_json(file_path)
 

    lst_responses = [] # list to collect responses from all models (per prompt)
    # Iterate through all prompts and get responses from all models
    print("Starting LLM API calls")
    for counter, row in enumerate(df_prompts.itertuples()):
        # Intialize the model configurations.
        prompt_config = ModelConfig(row.questions, my_api, row.temperature, row.top_p, row.max_tokens, row.seed, row.stream)
        
        # Sending a single prompt to all selected models (output is a list of dictionaries)
        full_llm_res = invoke_all_models(prompt_config, model_urls)
        
        # Collecting all responses from all models as a list of dictionaries for a single prompt.
        lst_responses.append(full_llm_res)
        print("Completed {} rounds".format(counter+1))

    
    lst_pd = [] # list of responses for all prompts to be read into pandas
    
    # Decompose list of list of dictionaries to a single list of dictionaries for dataframe processing 
    for sublist in lst_responses:
        for item in sublist:
            lst_pd.append(item)

    df_llm = pd.DataFrame(lst_pd) # Create the dataframe.
    # Add the classification to each model run.
    if file_path.endswith('.json'):
        model_len = len(df_llm['model_name'].unique())
        classes = ['short-in -> short-out'] * model_len + ['short-in -> long-out'] * model_len  + ['long-in -> short-out'] * model_len + ['long-in -> long-out'] * model_len
        df_llm['class'] = classes

    return df_llm
        

In [ ]:
def create_stats_llm(llm_df: pd.DataFrame) -> pd.DataFrame:
    """
    Creates a data frame that is a merge of two pandas groupby 
    dataframes of response time and output token.

    Args: 
        llm_df (pd.DataFrame): Pandas dataframe of LLM responses.

    Outputs:
        stats_df (pd.DataFrame): Pandas dataframe of LLM aggregation stats.
    
    """
    # Group by the response time for each model and take three statistical measures for response time.
    time_df = llm_df.groupby("model_name")['resp_time'].aggregate(resp_time_min='min',
                                                                  resp_time_max='max',resp_time_mean='mean')
    time_df = time_df.reset_index()
    time_df = time_df.sort_values('resp_time_mean', ascending=True).reset_index(0, drop=True)

    # Group by the output tokens for each model and take three statistical measures for output tokens.
    tkn_df = llm_df.groupby("model_name")['out_tokens'].aggregate(out_tkn_min="min", out_tkn_max="max", out_tkn_mean="mean")
    tkn_df = tkn_df.reset_index()
    tkn_df = tkn_df.sort_values("out_tkn_mean", ascending=True).reset_index(0, drop=True)

    # Merge both dataframes into one dataframe for ease of use.
    stats_df = time_df.merge(tkn_df, how="inner", on="model_name")

    return stats_df
    

In [ ]:
def df_save(llm_df: pd.DataFrame, stats_llm: pd.DataFrame) -> None:
    """
    Saves two data frames to separate json files.

    Args:
        llm_df (pd.DataFrame): pandas dataframe of LLM responses
        stats_llm (pd.DataFrame): pandas dataframe of LLM aggregation stats.
        
    """
    # Create a folder to store the outputs.
    folder_path = "./output_llm/"
    os.makedirs(folder_path, exist_ok = True)

    # Saving the inputs from llm_results and stats_llm.
    llm_df.to_json(folder_path + "results_llm.json", orient = "records", compression = "infer")

    stats_llm.to_json(folder_path + "stats_llm.json", orient = "records", compression = "infer")


### Sending the prompt request for all prompt data over all models 
Below this cell, it will call the above dataframe functions to create the 
data structure we need to get the statistical info concerning the models used

In [ ]:
# The CSV file has five rows of prompt data.
csv_path = base_file_path + csv_prompt_file # Setting the csv path.

start_tm = time.time() 
# Create the LLM results dataframe. (#prompts x #models)
df_llm = create_llm_dataset(csv_path, model_urls)

# Create the Statistical LLM dataframe. (#models x 6-statistical measures)
llm_stats = create_stats_llm(df_llm)

end_tm = time.time() 
elapsed_time = (end_tm - start_tm) / 60
print("Elapsed time to run benchmark: {} minutes".format(round(elapsed_time,3)))

# Save the data structures to json.
df_save(df_llm, llm_stats)

In [ ]:
# Show the benchmark dataframe for the LLM models.
print("Benchmark statistics for the LLM Models.")
print(f"Shape of llm_stats dataframe is {llm_stats.shape}")
llm_stats

In [ ]:
print("Snapshot of the LLM Results from multiple prompts.")
df_llm.head(15)

# 2. Prompt scenario performance experiment.

Now, it is time to the feed json data containing four scenarios pretraining to the length of the input and output tokens.
The scenarios are listed as follows:
 - short input -> short output
 - short input -> long output
 - long input -> short output
 - long input -> long output

**NOTE**: There appears to be an issue if max_tokens is set above 1024, it returns a 422 HTTP Error. 


In [ ]:
# Initialize json file path. This file contains four JSON items.
json_path = base_file_path + json_prompt_file

# Create the LLM results dataframe for the scenarios. Dataframe size is (4 X #models) x 6 data columns 
start_time = time.time()
df_llm_kv = create_llm_dataset(json_path, model_urls)
end_time = time.time()

elapsed_time_f = (end_time - start_time) / 60
print("Elapsed time to run benchmark: {} minutes".format(round(elapsed_time_f,3)))
print("Rows: {}, Columns: {}".format(df_llm_kv.shape[0], df_llm_kv.shape[1]))

In [ ]:
# Inspect the dataframe to check for the validity of the dataframe transformation.
df_llm_kv.head(10)

# Charts of the results.
Let us take a look at the results through Matplotlib. The cells below will display seven
different plots of the response time vs input and output tokens and complete tokens (input and output tokens).

### Initialize plots variables.

In [ ]:
# Copy the dataframe to a variable to use in plotting.
df = df_llm_kv

## Defining plot values
markers = [
    "o",  # Circle
    "s",  # Square
    "^",  # Triangle Up
    "P",  # Plus (filled)
    "*",  # Star
    "$\clubsuit$",  # Club symbol using LaTeX
    "X",  # X (filled)
    "D",  # Diamond
    "p",  # Pentagon
    "H",  # Hexagon
    "v",  # Triangle Down
    "+"   # Plus
]

marker_size = 100  # Size of markers

# Set the colors magenta, light green, orangered, and blue into a numpy array
colors = np.array([[1.0, 0.0, 1.0, 1.0],
                   [0.0, 0.5, 0.0, 1.0],
                   [1.0, 0.27, 0.0, 1.0],
                   [0.0, 0.0, 1.0, 1.0]])


# Unique values for model_name and class
unique_models = df['model_name'].unique()
unique_classes = df['class'].unique()

# Markers and colors to be used in the subplots.
m_subplots = markers[:len(unique_models)]
c_subplots = ['magenta', 'green', 'orangered', 'blue']

# Line style and width for grid lines.
line_style, line_width = "--", 1.0

## Subplots of Output Tokens vs Response Time

In [ ]:
# Create 2x2 subplots
fig, axs = plt.subplots(2, 2, figsize=(16, 12), sharex=True, sharey=True)
axs = axs.flatten()

# Loop through each class for plotting
for i, class_value in enumerate(unique_classes):
    ax = axs[i]
    subset = df[df['class'] == class_value]
    
    # Plot each model with a different marker
    for j, model in enumerate(unique_models):
        model_subset = subset[subset['model_name'] == model]
        ax.scatter(model_subset['out_tokens'], model_subset['resp_time'], 
                   marker=m_subplots[j], color=c_subplots[i], label=model, s=marker_size)
    
    # Label the axes
    ax.set_title(f'Query Type: {class_value}', fontsize=14)
    ax.grid(axis='both', linewidth = line_width, linestyle = line_style)
    ax.set_xlabel('Number of Output Tokens')
    ax.set_ylabel('Response Time secs')

legend_elements = [Line2D([0], [0], marker=markers[i], color='w', label=unique_models[i],
                          markerfacecolor='k', markersize=12) for i in range(len(unique_models))]
fig.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1.1, 1), title='Model')
plt.savefig('output_llm/response_time-vs-out_tokens-subplots.png')
plt.show()

## Subplots of Input Tokens vs Response Time

In [ ]:
# Create 2x2 subplots
fig, axs = plt.subplots(2, 2, figsize=(16, 12), sharex=True, sharey=True)
axs = axs.flatten()

# Loop through each class for plotting
for i, class_value in enumerate(unique_classes):
    ax = axs[i]
    subset = df[df['class'] == class_value]
    
    # Plot each model with a different marker
    for j, model in enumerate(unique_models):
        model_subset = subset[subset['model_name'] == model]
        ax.scatter(model_subset['in_tokens'], model_subset['resp_time'], 
                   marker=m_subplots[j], color=c_subplots[i], label=model, s=marker_size)
    
    # Label the axes
    ax.set_title(f'Query Type: {class_value}', fontsize=14)
    ax.grid(axis='both', linewidth = line_width, linestyle = line_style)
    ax.set_xlabel('Number of Input Tokens')
    ax.set_ylabel('Response Time secs')

legend_elements = [Line2D([0], [0], marker=markers[i], color='w', label=unique_models[i],
                          markerfacecolor='k', markersize=12) for i in range(len(unique_models))]

fig.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1.1, 1), title='Model')
plt.savefig('output_llm/response_time-vs-in_tokens-subplots.png')
plt.show()

## Response time vs number of input tokens across all models

In [ ]:
plt.figure(figsize=(12, 8)) # Create a scatter plot

# Loop over each unique model_name
for i, model in enumerate(df['model_name'].unique()):
    # Subset the DataFrame based on model_name
    subset = df[df['model_name'] == model]
    
    # Loop over each unique prompt_id within the subset
    for j, prompt_id in enumerate(subset['class'].unique()):
        prompt_subset = subset[subset['class'] == prompt_id]
        
        # Plot using a different marker and color for each prompt_id
        plt.scatter(prompt_subset['in_tokens'], prompt_subset['resp_time'], label=f"{model} - query-type: {prompt_id}",
                    marker=markers[i % len(markers)], color=colors[j % len(colors)], s=marker_size)

# Labeling axes and title
plt.xlabel('Number of Input Tokens')
plt.ylabel('Response Time (seconds)')
plt.title('Response Time vs. Number of Input Tokens Across All Models')

# Place the legend outside the plot on the right side
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), title='Model - Prompt Type')
plt.grid(axis='x', linewidth = 0.5, linestyle = '--')
plt.grid(axis='y', linewidth = 0.5, linestyle = '--')
plt.tight_layout()  # Adjust layout to not cut off the legend
plt.savefig('output_llm/response_time-vs-in_tokens-all.png') # Save the image.
plt.show()

# Response time vs number of output tokens across all models

In [ ]:
# Create a scatter plot
plt.figure(figsize=(12, 8))

# Loop over each unique model_name
for i, model in enumerate(df['model_name'].unique()):
    # Subset the DataFrame based on model_name
    subset = df[df['model_name'] == model]
    
    # Loop over each unique prompt_id within the subset
    for j, prompt_id in enumerate(subset['class'].unique()):
        prompt_subset = subset[subset['class'] == prompt_id]
        
        # Plot using a different marker and color for each prompt_id
        plt.scatter(prompt_subset['out_tokens'], prompt_subset['resp_time'], label=f"{model} - query-type {prompt_id}",
                    marker=markers[i % len(markers)], color=colors[j % len(colors)], s=marker_size)

# Labeling axes and title
plt.xlabel('Number of Output Tokens')
plt.ylabel('Response Time (seconds)')
plt.title('Response Time vs. Number of Output Tokens Across All Models')

# Place the legend outside the plot on the right side
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), title='Model - Prompt Type')

plt.tight_layout()  # Adjust layout to not cut off the legend
plt.grid(axis='x', linewidth = 0.5, linestyle = '--')
plt.grid(axis='y', linewidth = 0.5, linestyle = '--')
plt.savefig('output_llm/response_time-vs-out_tokens-all.png') # Save the image.
plt.show()

# 3. Gradio Interface

This section will present the functions of the LLMs as an interface through radio.

1. Users will select which model to use which allows them to enter a prompt to return 
   the result.

2. The function that invokes all the LLM will present the fastest models from top to bottom
   as a quick visual as to which model is best for the particular prompt.

**NOTE**: Functions had to be re-written to work with Gradio so that there are two functions in the notebook 
          that function similarly.



### Gradio Functions

In [ ]:
def llm_invoke_all(prompt:str):
  """
    This function will call from any model within the dictionary from Nvidia
    AI foundational models and run the model. This is strictly focused on
    the text-to-text models found on the link below:

    https://catalog.ngc.nvidia.com/orgs/nvidia/teams/ai-foundation/models

    Models that require context are not included in the dictionary.

    Args:
        prompt (str) -> Prompt to be passed to the model.

    Outputs
        msg (str) -> text generated from the model given the prompt.
        resp_time (str) -> time taken to generate the response.
        input_tokens (int) -> number of tokens generated by the user input.
        out_tokens (int) -> number of tokens produced by the LLM.
        model_name (str) -> name of the model that was called by the curl request.
        prompt (str) -> prompt used to generate the output.

  """

  headers = {
    "Authorization": "Bearer " + str(my_api),
    "Accept": "application/json",
  }

  payload = {
  "messages": [
    {
      "content": str(prompt),
      "role": "user"
    }
  ],
  "temperature": 0.2,
  "top_p": 0.7,
  "max_tokens": 1024,
  "seed": 42,
  "stream": False
  }

  lst_resp = [] # Initialize list for the main output.
    
  #Create session.
  session = requests.Session()

  # Issue: function had to be rewritten to solve None results from output_tokens.   
  for key, url in model_urls.items():
    tmp_dict = {}
    response = session.post(url, headers = headers, json = payload)

    while response.status_code == 202:
      request_id = response.headers.get("NVCF-REQID")
      fetch_url = fetch_url_format + request_id
      response = session.get(fetch_url, headers=headers)

    response.raise_for_status()
    response_body = response.json()
    msg = response_body.get('choices')[0].get('message').get('content')
    resp_time = round(response.elapsed.total_seconds(), 3)
    out_tokens = response_body.get('usage').get('completion_tokens')
    in_tokens = response_body.get('usage').get('prompt_tokens')
    tmp_dict = {"model_name": key,"resp_time": resp_time, "out_tokens": out_tokens,
                "in_tokens": in_tokens, "prompt": prompt, "resp_msg": msg}
    lst_resp.append(tmp_dict)
    time.sleep(0.2)

  return lst_resp

def llm_invoke(model_name:str, prompt:str):
    
    """
    This function will call from any model within the dictionary from Nvidia
    AI foundational models and run the model. This is strictly focused on
    the text-to-text models found on the link below:

    https://catalog.ngc.nvidia.com/orgs/nvidia/teams/ai-foundation/models

    Models that require context are not included in the dictionary.

    Args:
        model_name (str): name of the model
        prompt (str): prompt to be passed to the model

    Outputs:
    Dictionary of the following outputs.
        model_name (str): name of the model
        resp_time (str): time taken to generate the response
        out_tokens (int): number of tokens returned from the LLM.
        in_tokens (int): number of tokens that represent the prompt.
        prompt (str): input string to be ingested by the LLM.
        msg (str): text generated from the model given the prompt

    """
    # Initialize the headers and payloads for the CURL requests.
    headers = {
        "Authorization": "Bearer " + str(my_api),
        "Accept": "application/json",
     }

    payload = {
        "messages": [
        {
            "content": str(prompt),
             "role": "user"
        }
    ],
    "temperature": 0.2,
    "top_p": 0.7,
    "max_tokens": 1024,
    "seed": 42,
    "stream": False
    }

    # Error catching if not model is not list of models.
    model_name = model_name.lower().replace(" ","")
    if model_name not in model_urls.keys():
        print("Model name not found in dictionary, using default model")
        print("Default model is NV-Llama2-70B-RLHF")
        model_name = "nv-llama2-70b-rlhf"
    model_url = model_urls[model_name]
    
    model_session = requests.Session()
    
    return ngc_request(model_session, model_url, headers, payload, model_name, prompt)

def gradio_all(prompt: str, save_as_file=False):
    """
    Formats the output to human readable style when a prompt is sent
    to all models

    Args
        prompt: string input from the user.
        save_as_file: boolean for the user to click checkbox to save the file.
    """
    content_lst = list() # Initialize an empty list.
    response_lst = llm_invoke_all(prompt) # Call the function. 
    response_lst = sorted(response_lst, key=lambda x: x['resp_time'], reverse = False) # Sort by response time.
    save_json(response_lst, save_as_file) # Call the save file function.
    for doc in response_lst:
        content = f"Model Name: {doc['model_name']}  Response Time:{doc['resp_time']} seconds\n  #Input Tokens:{doc['in_tokens']}  #Output Tokens:{doc['out_tokens']}\n\n"
        content_lst.append(content)
    return " ".join(content_lst)

def save_json(data, save_as_file):
    """
    Gives the option in Gradio to save output to JSON.

    Args
        data (dict | list): either a dictionary or a list of dictionaries
        save_as_file (bool): boolean for the user to click the checkbox to save the file.
    """
    
    json_data = json.dumps(data)
    if save_as_file:
        with open('./data/response_llm.json', 'w') as file:
            file.write(json_data)
        return "File Saved Successfully"
    else:
        return json_data

def gradio_one(model_name:str, prompt:str, save_as_file=False):
    """
    Formats the output to a human-readable style when a model is selected
    get a generated output.

    Args:
        model_name (str): model name that is being invoked
        prompt (str): string containing the human input text.
        save_as_file (bool): boolean for the user to click the xcheckbox to save the file.
    """
    # Call the model and format the output by calling the key values.
    llm_out = llm_invoke(model_name, prompt)
    
    output = f"{llm_out['message']}\n\nResponse time: {llm_out['resp_time']} seconds\n\n #Output Tokens:{llm_out['out_tokens']}  #Input Tokens:{llm_out['in_tokens']}\n  Model Name:{llm_out['model_name']}"
    save_json(llm_out, save_as_file)
    return output

### Gradio Interface: Single Model with prompt with option to save result.

In [ ]:
iface_singular = gr.Interface(fn = gradio_one,
                     inputs =[gr.Radio(list(model_full_name.values()), label="Model Name"),
                              gr.Textbox(label="Enter your Prompt"),
                              gr.Checkbox(label="Save as File")],
                     outputs = gr.Textbox(label="LLM Output"),
                     title = "Nvidia LLM Invoker",
                     description = "Choose a model and enter a prompt to invoke a LLM model from Nvidia AI Foundational Models.")

iface_singular.launch()

In [ ]:
iface_singular.close()

### Gradio Interface: Single Prompt across all models with option to save result.

In [ ]:
iface_llm = gr.Interface(fn = gradio_all,
                         inputs = [gr.Textbox(label="Enter your Prompt"),
                                   gr.Checkbox(label="Save as file")],
                         outputs = gr.Textbox(label="Best Performing LLMs with respect to time."),
                         title = "Nvidia Multi-Model Invoker",
                         description = "Enter a prompt to invoke multiple LLMs from Nvidia AI Foundational Models and return the response times."
)

iface_llm.launch()

In [ ]:
iface_llm.close()

# Conclusion and Additonal Work.

In this notebook, we were able to test multiple models from the NGC AI foundational models catalog against many prompts.
We were able to plot the results of the test to see how models perform in different scenarios as well as benchmark models
using statistical measures to see which were the best-performing models concerning response time.

Here is some additional work that can be done to improve this notebook:

* Refactor code so that the request function to send prompts to NGC models works with or without Gradio. The current code had to be separated into two separate functions that perform the same function.

* Extend the functionality to other types of models on the NVIDIA AI Foundation Model catalog. This notebook focused on text-to-text models where the model payloads were of the same structure.

* Add functionality to allow for async CURL requests to the API. Encountered two bugs when working the modules async and aiohttp. The explanation can be found at the top of the notebook.

* Extend the functionality to evaluate the metrics of each LLM for accuracy to different topics. Each request gives back a text response. Evaluating the speed of an LLM is easy, but accuracy is not and it is very dependent on the use case. Possibly leverage [AlpacaEval](https://github.com/tatsu-lab/alpaca_eval)

